* This is Part 4 of 4 Code files for Week 2 Titanic Modelling assignment
### Create the ypreds file for the Kaggle competition using my Random Forrest model 

In [18]:
import pandas as pd
import os
import numpy as np
#wd = os.getcwd()
wd = 'C:/Users/lisbo/OneDrive/AC Career/Spiced_Academy_DS/spiced_projects/cascabel-curve-student-code/week_02/data/'

df_Ktest = pd.read_csv(wd + 'test.csv', index_col=0, header =0)


In [19]:
""" Import the complete Titanic Training dataset from Kaggle ad fit a new and better  
Random forrest model and then generate better Ypred , do not test/train split buy instead use 
the Kaggle test data to test the model to produce the ypreds  """

wd = 'C:/Users/lisbo/OneDrive/AC Career/Spiced_Academy_DS/spiced_projects/cascabel-curve-student-code/week_02/data/'

df_K_rfmod = pd.read_csv(wd + 'train.csv', index_col=0, header =0)
df_K_rfmod = df_K_rfmod.reset_index()


In [20]:

#import the Random Forrest Model from file 3 
%store -r rf_mod
rf_mod = rf_mod


In [21]:
rf_mod

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=7, n_estimators=15)

In [22]:
#import the Logistic Model from file 2
%store -r lg_mod
lg_mod = lg_mod

lg_mod 

LogisticRegression(class_weight='balanced', max_iter=10000)

In [23]:
lg_mod 

LogisticRegression(class_weight='balanced', max_iter=10000)

In [24]:
###Usig KAGGLE TEST DATASET to create Predictors file for the Kaggle Titanic competition

In [25]:
####Passs the Kaggle test dataset 'df_Ktest' through the same data feature engineering as my 
##origninal Traiing dataset in order to prepare it properly for prediction generation

In [26]:
df_Ktest.columns


Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked'],
      dtype='object')

In [27]:

##assess the average ages of passengers based on their Sex and Pclass

df_Ktest_byPclass = df_Ktest.groupby(by=['Sex','Pclass'])
df_Ktest_byPclass = df_Ktest.groupby(by=['Sex','Pclass'])
df_Ktest_byPclass['Age'].mean()
#df_Ktest_byPclass['Age_i'].mean() 

#Impute the average age based on the average age by 'Sex' and 'Pclass'
df_Ktest_byPclass = df_Ktest.groupby(by=['Sex','Pclass'])
df_Ktest_byPclass['Age'].mean()

df_Ktest['C1'] = np.where(np.isnan(df_Ktest['Age'].values),1,0) 

def new_val(df_Ktest):    
    if  (df_Ktest['C1'] == 1 ) & (df_Ktest['Sex'] == 'female' ) & (df_Ktest['Pclass'] == 1)  :
        return 35
    elif (df_Ktest['C1'] == 1) & (df_Ktest['Sex'] == 'female')  & (df_Ktest['Pclass'] == 2)  :
        return 29
    elif (df_Ktest['C1'] == 1) & (df_Ktest['Sex'] == 'female' ) & (df_Ktest['Pclass'] == 3)  :
        return 21
    elif (df_Ktest['C1'] == 1) & (df_Ktest['Sex'] == 'male')  & (df_Ktest['Pclass'] == 1)  :
        return 4
    elif (df_Ktest['C1'] == 1) & (df_Ktest['Sex'] == 'male')  & (df_Ktest['Pclass'] == 2)   :
        return 31   
    elif (df_Ktest['C1'] == 1) & (df_Ktest['Sex'] == 'male')  & (df_Ktest['Pclass'] == 3 )  :
        return 27 
    else:
        return df_Ktest['Age']

df_Ktest['Age_i'] = df_Ktest.apply(new_val, axis = 1)

###create Age categoricla variable
df_Ktest['Age_i'] = df_Ktest['Age_i']
#.astype(int)

def new_acat(df_Ktest):    
    if  (df_Ktest['Age_i'] <= 12 )  :
        return 'Child'
    elif (df_Ktest['Age_i'] > 12) & (df_Ktest['Age_i'] <= 19)  :
        return 'Teenager'
    elif (df_Ktest['Age_i'] > 19) & (df_Ktest['Age_i'] <= 35 )  :
        return 'Young_Adults'
    elif (df_Ktest['Age_i'] > 35) & (df_Ktest['Age_i'] <= 60 )  :
        return 'Middle_Aged'
    elif (df_Ktest['Age_i'] > 60)   :
        return 'Elderly'   

df_Ktest['Age_Cat'] = df_Ktest.apply(new_acat, axis = 1)

##convert new age categories into dummy variables   
age_dums = pd.get_dummies(df_Ktest['Age_Cat'])
df_Ktest=df_Ktest.join(age_dums, lsuffix="_left", rsuffix="_right")

###
##create numeric Age categorical variable for he Randow Forrest data
df_Ktest['Age_Cat_i'] = df_Ktest['Age_Cat'].map({'Child':  0, 'Teenager':  1, 'Young_Adults': 2, 'Middle_Aged':  3, 'Elderly':  4, 'Deck_F': 5 })

#Extract the titles from the name field to 
import re #import the regular expresiona package

def create_title(name):
    
    # try to find the pattern, returns None if nothing is found
    title_search = re.search(' ([A-Za-z]+)\.', name)
    
    # return the title if exists
    if title_search:
        return title_search.group(1)
    
    else:
        return ""

# create the new title column
df_Ktest['Title'] = df_Ktest['Name'].apply(create_title)

#group and rename the less frequent Titles into a 'RareTitle' categrory value:
#Rename the followin titles:

replacements = {
   'Title': {
    'Dr': 'RareTitle', 
    'Major': 'RareTitle', 
    'Col': 'RareTitle', 
    'Mile': 'RareTitle', 
    'Capt': 'RareTitle', 
    'Ms': 'RareTitle', 
    'Jonkheer': 'RareTitle', 
    'Don': 'RareTitle', 
    'Sir': 'RareTitle', 
    'Lady': 'RareTitle',
    'Countess': 'RareTitle',
    'M': 'RareTitle',
    'Mme': 'RareTitle',
    'Rev': 'RareTitle',
    'Mlle': 'RareTitle'}
}

df_Ktest.replace(replacements,  inplace=True)

df_Ktest['Title'].unique()
df_Ktest['Title'].value_counts()
#convert the 'Title' column into a binary dummy set of variables for each unique categoy:
#'Mr', 'Mrs', 'Miss', 'Master', 'RareTitle', 'Rev', 'Mlle'

title_dums = pd.get_dummies(df_Ktest['Title'])
df_Ktest=df_Ktest.join(title_dums, lsuffix="_left", rsuffix="_right")

#also create a numeric Title_cat column for the Randowm Forrest Model:

def title_acat(df_Ktest):    
    if  (df_Ktest['Title'] == 'Mr' )  :
        return 0
    elif (df_Ktest['Title'] == 'Mrs')   :
        return 1
    elif (df_Ktest['Title'] == 'Miss')  :
        return 2
    elif (df_Ktest['Title'] == 'Master')   :
        return 3
    elif (df_Ktest['Title'] == 'RareTitle')   :
        return 4   

df_Ktest['Title_Cat'] = df_Ktest.apply(title_acat, axis = 1)
#.astype(int)

################################################


###Fare variable
# Import libraries


###create  new Fare categorical variable

def new_fcat(df_Ktest):    
    if  (df_Ktest['Fare'] < 10 )  :
        return 'Fare_lt10'
    elif (df_Ktest['Fare'] >= 10) & (df_Ktest['Fare'] <= 25)  :
        return 'Fare_btw_10_25'
    elif (df_Ktest['Fare'] > 25) & (df_Ktest['Fare'] <= 50 )  :
        return 'Fare_mt25_50'
    elif (df_Ktest['Fare'] > 50) & (df_Ktest['Fare'] <= 100 )  :
        return 'Fare_mt50_100'
    elif (df_Ktest['Fare'] > 100)   :
        return 'Fare_mt100'   

df_Ktest['Fare_Cat'] = df_Ktest.apply(new_fcat, axis = 1)

##make numeric 'Fare_Cat' column for Random Forrest model
df_Ktest['Fare_Cat'] = df_Ktest.apply(new_fcat, axis = 1)

df_Ktest['Fare_Cat_i'] = df_Ktest['Fare_Cat'].replace(['Fare_lt10', 'Fare_btw_10_25', 'Fare_mt25_50','Fare_mt50_100','Fare_mt100'],
                        [0, 1,2,3,4])
df_Ktest['Fare_Cat_i'] = df_Ktest['Fare_Cat_i']
#.astype(str)

#create dummy variable for Fare variable

df_Ktest['Fare_Cat'].unique()
df_Ktest['Fare_Cat'].value_counts()

fare_dums = pd.get_dummies(df_Ktest['Fare_Cat'])
df_Ktest=df_Ktest.join(fare_dums, lsuffix="_left", rsuffix="_right")

# Replace ['Embarked'] column NaN values with 'S' the most common value
df_Ktest['Embarked'] = df_Ktest['Embarked'].fillna('S')
# check if all values have been replaced
print("Remaining NaN values: {}".format(df_Ktest['Embarked'].isnull().sum()))

# convert 'Sex' and 'Embarked' into numeric variables for the model
# female equals 0 and male equals 1 in the sex column
df_Ktest['Sex'] =  df_Ktest['Sex'].map({'female': 0, 'male': 1})

# C = 0, Q = 1, S = 2 in the embarked column
df_Ktest['Embarked'] = df_Ktest['Embarked'].map({'C':  0, 'Q':  1, 'S': 2})

##Create a 'Deck' variable from the Cabin variable
import re #regex library for character search

deck = {"A": "Deck_A", "B": "Deck_B", "C": "Deck_C", "D": "Deck_D", "E": "Deck_E", "F": "Deck_F", "G": "Deck_G", "U": "U"}

df_Ktest['Cabin'] = df_Ktest['Cabin'].fillna("U0")
df_Ktest['Deck'] = df_Ktest['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
df_Ktest['Deck'] = df_Ktest['Deck'].map(deck)
df_Ktest['Deck'] = df_Ktest['Deck'].fillna(0) ##replace all the missing values with a 0
df_Ktest['Deck'] = df_Ktest['Deck'].astype(str)

##create 'Deck' dummies variable for the Logistic regression model
Deck_dums = pd.get_dummies(df_Ktest['Deck'])
df_Ktest=df_Ktest.join(Deck_dums, lsuffix="_left", rsuffix="_right")

##create numeric categorical variable for he Randow Forrest data
df_Ktest['Deck'] = df_Ktest['Deck'].map({'Deck_A':  0, 'Deck_B':  1, 'Deck_C': 2, 'Deck_D':  3, 'Deck_E':  4, 'Deck_F': 5,  'Deck_G':  6, 'U':  7    })

#df_Ktest.drop([df_Ktest.index[339]], inplace = True)
#df_Ktest.drop(['0'], axis=1, inplace=True)




Remaining NaN values: 0


In [28]:
df_Ktest2 = df_Ktest

In [29]:
#Select only relevant columns from the Kaggle test dataset for the Random Forrest Model:
df_Ktest.drop([ 'Name','Age', 'Age_Cat', 'Ticket', 'Fare', 'Cabin', 'C1', 'Age_i','Title', 'Fare_Cat','Child', 'Elderly', 'Middle_Aged', 'Teenager','Young_Adults', 'Title', 'Master', 'Miss', 'Mr', 'Mrs', 'RareTitle','Fare_Cat','Fare_btw_10_25', 'Fare_lt10','Fare_mt100', 'Fare_mt25_50', 'Fare_mt50_100', 
 'Deck_A','Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'U' ], axis=1, inplace=True)

In [30]:
##manaully tidy up the df_Ktest dataset
#these fixes are minor bugs from the feature engineering stage that need a manual solution

#df_Ktest.info()
df_Ktest.drop(['Dona'], axis=1, inplace=True)
df_Ktest['Fare_Cat_i'] = df_Ktest['Fare_Cat_i'].fillna(0)
df_Ktest['Title_Cat'] = df_Ktest['Title_Cat'].fillna(0)



In [31]:
K_predictions = rf_mod.predict(df_Ktest)
df_Kpred = pd.DataFrame(K_predictions, index=df_Ktest.index, columns=['Survived'])

df_Kpred.to_csv(r'Kaggle_Predictions3.csv', index = True)

In [32]:
K_predictions

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,

In [33]:
####ENd OF PROJECT ###

In [34]:
###use the logistic regression model

In [35]:
#Select only relevant columns from the Kaggle test dataset for the Random Forrest Model:
df_Ktest2 = df_Ktest2.drop([ 'Name','Age', 'Age_Cat', 'Ticket', 'Fare', 'Cabin', 'C1', 'Age_i','Title', 'Fare_Cat','Child', 'Elderly', 'Middle_Aged', 'Teenager','Young_Adults', 'Title', 'Master', 'Miss', 'Mr', 'Mrs', 'RareTitle','Fare_Cat','Fare_btw_10_25', 'Fare_lt10','Fare_mt100', 'Fare_mt25_50', 'Fare_mt50_100', 
 'Deck_A','Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'U' ], axis=1, inplace=True)



KeyError: "['Name' 'Age' 'Age_Cat' 'Ticket' 'Fare' 'Cabin' 'C1' 'Age_i' 'Title'\n 'Fare_Cat' 'Child' 'Elderly' 'Middle_Aged' 'Teenager' 'Young_Adults'\n 'Title' 'Master' 'Miss' 'Mr' 'Mrs' 'RareTitle' 'Fare_Cat'\n 'Fare_btw_10_25' 'Fare_lt10' 'Fare_mt100' 'Fare_mt25_50' 'Fare_mt50_100'\n 'Deck_A' 'Deck_B' 'Deck_C' 'Deck_D' 'Deck_E' 'Deck_F' 'Deck_G' 'U'] not found in axis"